In [1]:
import pandas
import numpy as np
import shutil
import matplotlib.pyplot as plt
import math
import os
import glob

In [2]:
ROOT_DIR="./Datasets/lung"
number_of_images={}
for dir in os.listdir(ROOT_DIR):
    number_of_images[dir]=len(os.listdir(os.path.join(ROOT_DIR,dir)))

In [3]:
print(number_of_images.items(),sep="\n")
os.environ['XLA_FLAGS'] = "--xla_gpu_cuda_data_dir=/usr/lib/cuda"
os.environ['TF_GPU_ALLOCATOR'] = "cuda_malloc_async"
os.system('echo $XLA_FLAGS')
os.system('echo $TF_GPU_ALLOCATOR')

dict_items([('Normal cases', 416), ('Malignant cases', 561)])
--xla_gpu_cuda_data_dir=/usr/lib/cuda
cuda_malloc_async


0

In [4]:
def dataFolder(path,split):
    if not os.path.exists("./"+path):
        os.mkdir("./"+path)
        for dir in os.listdir(ROOT_DIR):
            os.makedirs("./"+path+"/"+dir)
            for img in np.random.choice(a= os.listdir(os.path.join(ROOT_DIR,dir)),
                                    size=(math.floor(split*number_of_images[dir])),
                                    replace=False):
                O=os.path.join(ROOT_DIR,dir,img)
                D=os.path.join("./"+path+"/",dir)
                shutil.copy(O,D)
    else:
        print(path+" folder exists")

In [5]:
dataFolder("train",0.7)
dataFolder("val",0.5)
dataFolder("test",1)

train folder exists
val folder exists
test folder exists


In [6]:
from keras.layers import Conv2D,MaxPool2D,Dropout,Flatten,Dense,BatchNormalization,GlobalAvgPool2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
import keras

2023-09-19 00:06:32.196153: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-19 00:06:32.242453: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-19 00:06:32.756499: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
model =Sequential()
# model.add(Conv2D(filters=8,kernel_size=(3,3),activation='relu',input_shape=(512,512,3),padding='same'))
# model.add(Conv2D(16, 3, activation='relu'))
# model.add(MaxPool2D())
# model.add(Conv2D(16, 3, activation='relu'))
# model.add(MaxPool2D())
# model.add(Conv2D(16, 3, activation='relu'))
# model.add(MaxPool2D())
# model.add(Flatten())
# model.add(Dense(64, activation='relu'))
# model.add(Dense(units=1))
# # model.add(Dense(3))
# model.summary()
model.add(Conv2D(filters=16,kernel_size=(3,3),activation='relu',input_shape=(512,512,3),padding='same'))
model.add(Conv2D(filters=32,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=128,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(units=64,activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(units=2,activation='relu'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 512, 512, 16)      448       
                                                                 
 conv2d_1 (Conv2D)           (None, 510, 510, 32)      4640      
                                                                 
 max_pooling2d (MaxPooling2  (None, 255, 255, 32)      0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 253, 253, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 126, 126, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 124, 124, 128)     7

2023-09-19 00:06:33.369224: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-19 00:06:33.397632: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-19 00:06:33.397723: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-19 00:06:33.398852: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-19 00:06:33.398884: I tensorflow/compile

In [8]:
model.compile(optimizer='adam',loss=keras.losses.binary_crossentropy,metrics=['accuracy'])

In [9]:
train_data=ImageDataGenerator(zoom_range=0.2,shear_range=0.2,
                   horizontal_flip=True).flow_from_directory(directory='./train',target_size=(512,512),batch_size=32,class_mode='binary')
train_data.class_indices

Found 683 images belonging to 2 classes.


{'Malignant cases': 0, 'Normal cases': 1}

In [10]:
test_data=ImageDataGenerator().flow_from_directory(
    directory='./test',target_size=(512,512),batch_size=32,class_mode='binary')
val_data=ImageDataGenerator().flow_from_directory(
    directory='./val',target_size=(512,512),batch_size=32,class_mode='binary')

Found 977 images belonging to 2 classes.
Found 488 images belonging to 2 classes.


In [11]:
from keras.callbacks import ModelCheckpoint,EarlyStopping
es=EarlyStopping(monitor="val_accuracy",min_delta=0.01,patience=4,verbose=1,mode='auto')
mc=ModelCheckpoint(monitor="val_accuracy",filepath="./bestmodel.h5",verbose=1,save_best_only=True,mode='auto')
cd=[es,mc]

In [12]:
hs=model.fit(train_data,
                       steps_per_epoch=16,
                       epochs=10,verbose=1,
                       validation_data=val_data,validation_steps=16,
                       callbacks=cd)

Epoch 1/10


2023-09-19 00:06:35.810718: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-09-19 00:06:37.401624: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-09-19 00:06:46.973451: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f6682cd3850 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-19 00:06:46.973495: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 SUPER, Compute Capability 7.5
2023-09-19 00:06:46.976569: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-19 00:06:47.056418: I ./tensorflow/compiler/jit/device_compiler.h:186

16/16 [==============================] - ETA: 0s - loss: 7.1375 - accuracy: 0.5397
Epoch 1: val_accuracy improved from -inf to 0.57377, saving model to ./bestmodel.h5


/home/star/miniconda3/envs/tf/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


16/16 [==============================] - 72s 1s/step - loss: 7.1375 - accuracy: 0.5397 - val_loss: 6.5746 - val_accuracy: 0.5738
Epoch 2/10
16/16 [==============================] - ETA: 0s - loss: 6.7857 - accuracy: 0.5601
Epoch 2: val_accuracy did not improve from 0.57377
16/16 [==============================] - 14s 857ms/step - loss: 6.7857 - accuracy: 0.5601 - val_loss: 6.5746 - val_accuracy: 0.5738
Epoch 3/10
16/16 [==============================] - ETA: 0s - loss: 6.3145 - accuracy: 0.5906
Epoch 3: val_accuracy did not improve from 0.57377
16/16 [==============================] - 14s 844ms/step - loss: 6.3145 - accuracy: 0.5906 - val_loss: 6.5746 - val_accuracy: 0.5738
Epoch 4/10
16/16 [==============================] - ETA: 0s - loss: 6.5813 - accuracy: 0.5723
Epoch 4: val_accuracy did not improve from 0.57377
16/16 [==============================] - 14s 909ms/step - loss: 6.5813 - accuracy: 0.5723 - val_loss: 6.5746 - val_accuracy: 0.5738
Epoch 5/10
16/16 [======================

In [13]:
from keras.preprocessing.image import load_img,img_to_array

In [18]:
path="./Datasets/lung/Malignant cases/Malignant case (32).jpg"
img=load_img(path,target_size=(512,512))
arr=img_to_array(img)
arr.shape
arr=np.expand_dims(arr,axis=0)
pred=model.predict(arr)
pred_1=list(pred[0])
print(pred)
print(pred_1)
ans=list(train_data.class_indices.keys())
print(ans)
print(ans[pred_1.index(max(pred_1))])

1/1 [==============================] - 0s 14ms/step
[[0. 0.]]
[0.0, 0.0]
['Malignant cases', 'Normal cases']
Malignant cases
